# "MAXIMUM BETWEENNESS", 
Problem "MAXIMUM BETWEENNESS" je NP-težak problem sa kojim ću se u ovom projektu boriti korišćenjem sirovih tehnika optimizacije

Zadatak glasi:

Imamo skup A i kolekciju C uređenih trojki (a, b, c) različitih elemenata iz A. 
Cilj je pronaći jednoznačnu funkciju f koja preslikava elemente iz A u opseg od 1 do veličine skupa A, tako da se maksimalizuje broj trojki u kojima se poštuje određeni redosled (f(a)<f(b)<f(c) ili f(c)<f(b)<f(a)).

U nastavku ću da razvijem optimizacione metode, kao što su genetski algoritmi, simulirano kaljenje, ili lokalnu pretragu.
(Važno je da osiguram da moje metode mogu rukovati velikim skupovima podataka i da daju dobre rezultate).


In [2]:
import random
import math
from itertools import permutations

# Brute Force
Brute-force algoritam treba da mi obezbedi uporednu analizu optimizacionih metoda. 
On bi trebalo da bude u stanju da efikasno obrađuje male instance problema i da daje optimalna rešenja kako bi se mogla proveriti ispravnost optimizacionih pristupa.

In [15]:
def brute_force(A, C):
    max_betweenness = 0
    permutations_f = permutations(A)

    for perm in permutations_f:
        f = dict(zip(A, perm))

        current_betweenness = 0
        for (a, b, c) in C:
            if (f[a] < f[b] < f[c]) or (f[c] < f[b] < f[a]):
                current_betweenness += 1
                
        if(current_betweenness > max_betweenness):
            max_betweenness = current_betweenness
            max_f = f.values()

    
    return max_betweenness, max_f

# BNB

In [3]:
def brute_force_BNB(A, C):
    max_betweenness = 0
    C_size = len(C)

    permutations_f = permutations(A)
    for perm in permutations_f:
        f = dict(zip(A, perm))

        current_betweenness = 0
        i=0
        for (a, b, c) in C:
            i += 1
            if (f[a] < f[b] < f[c]) or (f[c] < f[b] < f[a]):
                current_betweenness += 1
            ostatak = C_size - i  
            if (current_betweenness + ostatak <= max_betweenness):
                    break

        if(current_betweenness > max_betweenness):
            max_betweenness = current_betweenness 
            max_f = f.values()
    
    return max_betweenness, max_f

### Pomoćne funkcije

In [4]:
# pomocne funkcije koje cemo koristiti u narednim algoritmima

def count_triplets(solution, C):
    count = 0
    for (a, b, c) in C:
        index_a = solution.index(a)
        index_b = solution.index(b)
        index_c = solution.index(c)
        if (index_a < index_b < index_c) or (index_c < index_b < index_a):
            count += 1
    return count

def initial_solution(A):
    return random.sample(A, len(A))


# Local Search

In [5]:
def neighborhood(solution):
    for i in range(len(solution)):
        for j in range(i+1, len(solution)):
            neighbor = solution[:]   #kopija
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            yield neighbor

def local_search_first_improvement(A, C, max_iterations=1000):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)

    for _ in range(max_iterations):
        past_solution = current_solution
        for neighbor in neighborhood(current_solution):
            neighbor_score = count_triplets(neighbor, C)
            if neighbor_score > current_score:
                current_solution = neighbor
                current_score = neighbor_score
                break  # cim naletimo na prvo bolje resenje
        if current_score == len(C):
            break
        if past_solution is current_solution:
            break
    
    return current_solution, current_score

def local_search_best_improvement(A, C, max_iterations=1000):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)
    best_solution = current_solution[:]
    best_score = current_score
    
    for _ in range(max_iterations):
        for neighbor in neighborhood(current_solution):
            neighbor_score = count_triplets(neighbor, C)
            if neighbor_score > best_score:
                best_solution = neighbor
                best_score = neighbor_score

        if(current_solution == best_solution):
            break;
        current_solution = best_solution
        current_score = best_score
        
    return current_solution, current_score

# Simultated Annealing

Simulirano kaljenje je metaheuristička tehnika koja simulira proces kaljenja metala. Ova tehnika ne zahteva iscrpnu pretragu celokupnog prostora pretrage, već iterativno unapređuje trenutno rešenje. Simulirano kaljenje može biti korisno kod velikih i složenih prostora pretrage.

In [5]:
def generate_neighbor(solution):
    neighbor = solution[:]
    i, j = random.sample(range(len(solution)), 2)
    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor

In [7]:
def simulated_annealing(A, C, initial_temperature=100, cooling_rate=0.95, stopping_temperature=1e-6, max_iterations=1000):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)
    best_solution = current_solution
    best_score = current_score
    temperature = initial_temperature
    
    for iteration in range(max_iterations):
        if temperature < stopping_temperature:
            break
        
        neighbor = generate_neighbor(current_solution)
        neighbor_score = count_triplets(neighbor, C)
        
        delta_score = neighbor_score - current_score

                                                    # Metropolisov kriterijum
        if delta_score > 0 or random.uniform(0, 1) < math.exp(delta_score / temperature):
            current_solution = neighbor
            current_score = neighbor_score
        
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score
        
        temperature *= cooling_rate
    
    return best_solution, best_score

Kada je temperatura visoka, Metropolisov kriterijum je veći, što znači da je veća verovatnoća prihvatanja lošijeg rešenja. Kako temperatura opada tokom vremena, verovatnoća prihvatanja lošijeg rešenja takođe opada, što omogućava algoritmu da se postepeno fokusira na poboljšavanje kvaliteta rešenja.

In [8]:
# Primeri strategija hlađenja temperature
def geometric_cooling(initial_temperature, iteration, cooling_rate):
    return initial_temperature * cooling_rate ** iteration

def sqrt_cooling(initial_temperature, iteration, cooling_rate):
    return initial_temperature * cooling_rate / math.sqrt(iteration + 1)

def log_cooling(initial_temperature, iteration, cooling_rate):
    return initial_temperature * cooling_rate / math.log(iteration + 2)

def simulated_annealing_cooling_strategy(A, C, cooling_strategy, initial_temperature=100, cooling_rate = 0.95, stopping_temperature=1e-6, max_iterations=1000):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)
    best_solution = current_solution
    best_score = current_score
    temperature = initial_temperature
    
    for iteration in range(max_iterations):
        if temperature < stopping_temperature:
            break
        
        neighbor = generate_neighbor(current_solution)
        neighbor_score = count_triplets(neighbor, C)
        
        delta_score = neighbor_score - current_score

        if delta_score > 0 or random.uniform(0, 1) < math.exp(delta_score / temperature):
            current_solution = neighbor
            current_score = neighbor_score
        
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score
        
        temperature = cooling_strategy(initial_temperature, iteration, cooling_rate)
    
    return best_solution, best_score

In [17]:
def adaptive_cooling(initial_temperature, iteration, acceptance_rate, target_acceptance_rate, cooling_rate):
    if acceptance_rate > target_acceptance_rate:
        return initial_temperature * cooling_rate
    else:
        return initial_temperature / math.log(iteration + 2)

def simulated_annealing_adaptive(A, C, initial_temperature=100, stopping_temperature=1e-6, max_iterations=1000, target_acceptance_rate=0.5, cooling_rate = 0.95):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)
    best_solution = current_solution
    best_score = current_score
    temperature = initial_temperature
    acceptance_count = 0
    total_count = 0
    
    for iteration in range(max_iterations):
        if temperature < stopping_temperature:
            break
        
        neighbor = generate_neighbor(current_solution)
        neighbor_score = count_triplets(neighbor, C)
        
        delta_score = neighbor_score - current_score

        if delta_score > 0 or random.uniform(0, 1) < math.exp(delta_score / temperature):
            current_solution = neighbor
            current_score = neighbor_score
            acceptance_count += 1
        
        total_count += 1
        acceptance_rate = acceptance_count / total_count
        temperature = adaptive_cooling(initial_temperature, iteration, acceptance_rate, target_acceptance_rate, cooling_rate)
    
    return best_solution, best_score


"adaptive_cooling" funkcija smanjuje temperaturu ako je stopa prihvatanja novih rešenja veća od ciljane stope prihvatanja, inače je lagano povećava.

In [4]:
def generate_neighbor_with_heuristic(C,solution,num_neighbors_to_generate):
    best_neighbor = None
    best_neighbor_score = -1
    for _ in range(num_neighbors_to_generate):
        neighbor = generate_neighbor(solution)
        neighbor_score = count_triplets(neighbor, C)
        if neighbor_score > best_neighbor_score:
            best_neighbor = neighbor
            best_neighbor_score = neighbor_score
    return best_neighbor

def simulated_annealing_with_intensive_search(A, C, initial_temperature=100, cooling_rate=0.95, stopping_temperature=1e-6, max_iterations=1000, num_neighbors_to_generate=10):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)
    best_solution = current_solution
    best_score = current_score
    temperature = initial_temperature
    
    for iteration in range(max_iterations):
        if temperature < stopping_temperature:
            break
        
        neighbor = generate_neighbor_with_heuristic(C ,current_solution, num_neighbors_to_generate)
        neighbor_score = count_triplets(neighbor, C)
        
        delta_score = neighbor_score - current_score

        if delta_score > 0 or random.uniform(0, 1) < math.exp(delta_score / temperature):
            current_solution = neighbor
            current_score = neighbor_score
        
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score
        
        temperature *= cooling_rate
    
    return best_solution, best_score


# Genetic Algorithm

Genetski algoritam je inspirisan procesom evolucije u prirodi. Koristi populaciju rešenja i primenjuje genetske operatore poput selekcije, ukrštanja i mutacije kako bi se pronašlo optimalno rešenje. Ova tehnika je posebno korisna za velike prostore pretrage i kompleksne probleme optimizacije.

Reprezentacija rešenja: Svako rešenje u prostoru rešenja predstavlja se kao jedna jedinka ili jedna jedinica.

Selekcija: Odabir roditelja iz populacije rešenja. Ovo se obično radi na osnovu funkcije cilja, tako da su rešenja sa boljim prilagođavanjem (višim vrednostima funkcije cilja) verovatnije izabrana za reprodukciju.

Ukrštanje: Kreiranje novih jedinki kombinovanjem genetskih informacija odabranih roditelja. Ovaj proces simulira ukrštanje genetskih informacija u prirodi.

Mutacija: Introdukcija slučajnih promena u jedinke kako bi se održala raznovrsnost populacije i izbegla zaglavljivanje u lokalnim optimumima.

Evaluacija: Procena svake jedinke u populaciji na osnovu funkcije cilja.

Ponavljanje procesa: Ponavljanje selekcije, ukrštanja i mutacije kroz generacije dok se ne dostigne određeni kriterijum zaustavljanja.

In [10]:
def initial_population(A, population_size):
    return [random.sample(A, len(A)) for _ in range(population_size)]

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + [gene for gene in parent2 if gene not in parent1[:crossover_point]]
    child2 = parent2[:crossover_point] + [gene for gene in parent1 if gene not in parent2[:crossover_point]]
    return child1, child2

def mutation(solution, mutation_rate):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(solution)), 2)
        solution[i], solution[j] = solution[j], solution[i]
    return solution

def genetic_algorithm(A, C, population_size=100, generations=1000, crossover_rate=0.8, mutation_rate=0.2):
    population = initial_population(A, population_size)
    
    for generation in range(generations):
        scores = [count_triplets(solution, C) for solution in population]
        
        selected_parents = random.choices(population, weights=scores, k=population_size)
        
        new_population = []
        for i in range(0, population_size, 2):
            if random.random() < crossover_rate:
                child1, child2 = crossover(selected_parents[i], selected_parents[i+1])
                new_population.extend([child1, child2])
            else:
                new_population.extend([selected_parents[i], selected_parents[i+1]])
        
        mutated_population = [mutation(solution, mutation_rate) for solution in new_population]
        
        population = mutated_population

    scores = [count_triplets(solution, C) for solution in population]
    
    best_solution_index = scores.index(max(scores))
    best_solution = population[best_solution_index]
    best_score = scores[best_solution_index]
    
    return best_solution, best_score